In [1]:
# Loading various libraries
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from matplotlib import pyplot as plt
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model 
from sklearn.model_selection import train_test_split

In [8]:
def read_files(file):
  notes=[]
  notes_to_parse=None
  # Parsing the midi file
  midi_file=converter.parse(file)
  # Seperating all the instruments from the file
  instr=instrument.partitionByInstrument(midi_file)

  for part in instr.parts:
    # Obtaining data only of Piano instrument
    if 'Piano' in str(part):
      notes_to_parse=part.recurse()

      # Iterating over all the parts of sub stream elements to check if element's type is Note or Chord. If it is Chord, split them into notes.
      for element in notes_to_parse:
        if type(element)==note.Note:
          notes.append(str(element.pitch))
        elif type(element)==chord.Chord:
          notes.append('.'.join(str(n) for n in element.normalOrder))

  # Returning all the notes in one file
  return notes

# Calling files recursively from the directory
file_path=["mozart"]
all_files=glob.glob("D:/TABITHA/InnoRave/automatic-music-generation-codes/" + 'All Midi Files/'+file_path[0]+'/*.mid',recursive=True)

# Collecting all the notes from each midi file in directory
notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)], dtype=object)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.40s/it]


In [18]:
# Unique Notes
notes1 = sum(notes_array,[]) 
unique_notes = list(set(notes1))
print("Unique Notes:",len(unique_notes))

# Mapping Notes with their Frequency
freq=dict(map(lambda x: (x,notes1.count(x)),unique_notes))

print("\nFrequency : No of Notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

Unique Notes: 147

Frequency : No of Notes
30 : 50
50 : 32
70 : 27
90 : 25


In [19]:
# Filtering notes greater than threshold frequency=50 Hz
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

# Creating new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

In [22]:
# Dictionary having 'note index:note'
ind_note=dict(enumerate(freq_notes))

# Dictionary having 'note:note index'
note_ind=dict(map(reversed,ind_note.items()))

In [23]:
timesteps=50

# Storing values of input(x) and output(y)
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    # Input will be the current index + timestep. Output will be the next index after timestep.
    inp=i[j:j+timesteps] 
    out=i[j+timesteps]

    # Appending the index value of respective notes 
    x.append(list(map(lambda x:note_ind[x],inp)))
    y.append(note_ind[out])

x_new=np.array(x) 
y_new=np.array(y)

In [24]:
# Reshaping input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

# Spliting the input into 80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [25]:
# Creating the model with two stacked LSTM layers with the latent dimension of 256
model = Sequential()
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))

# Fully connected layer for the output with softmax activation
model.add(Dense(len(note_ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 32)                8224      
                                                                 
Total params: 863,520
Trainable params: 863,520
Non-trai

In [ ]:
# Compiling the model using Adam optimizer
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Training the model on training sets and validating on testing sets
output = model.fit(x_train,y_train,
                   batch_size=512,epochs=120, 
                   validation_data=(x_test,y_test))

In [ ]:
# Saving the model
model.save("trained")

In [ ]:
# Load the model
model=load_model("trained")

# Generate random index
index = np.random.randint(0,len(x_test)-1)

# Get the data of generated index from x_test
music_pattern = x_test[index]

# For storing the predicted notes
out_pred_notes=[]

# Iterate till 200 notes are generated
for i in range(200):
  # Reshape the music pattern 
  music_pattern = music_pattern.reshape(1,len(music_pattern),1)
  
  # Get the maximum probability value from the predicted output
  pred_index = np.argmax(model.predict(music_pattern))

  # Get the note using predicted index and append to the output prediction list
  out_pred_notes.append(ind_note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)
  
  # Update the music pattern with one timestep ahead
  music_pattern = music_pattern[1:]

In [ ]:
output_notes = []
for offset,pattern in enumerate(out_pred_notes):
  # If pattern is a chord instance, split notes from the chord
  if ('.' in pattern) or pattern.isdigit():
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        # Cast the current note to Note object and append the current note 
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
    
    #cast the current note to Chord object. Offset will be 1 step ahead from the previous note as it will prevent notes to stack up 
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  
  else:
    # Cast the pattern to Note object, apply the offset and append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

# Save the midi file 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='Mozart-512,120.mid')

In [ ]:
# Plotting the graph of "Accuracy"
plt.plot(output.history['accuracy'])
plt.plot(output.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Plotting the graph of "Loss"
plt.plot(output.history['loss'])
plt.plot(output.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()